In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [ ]:
!pip install unsloth
!pip install transformers peft accelerate safetensors

from unsloth import FastLanguageModel, FastModel
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
True
Tesla T4


# Define Model

unsloth model straight from checkpoint

In [ ]:

model_a, tokenizer_a = FastLanguageModel.from_pretrained(
    model_name = '/content/drive/MyDrive/DPO/DPO on Colab/lora_adapter',
    max_seq_length = 2048,
)

FastLanguageModel.for_training(model_a)

AttributeError: 'NoneType' object has no attribute 'lower'

In [ ]:
model_test, tokenizer_test = FastLanguageModel.from_pretrained()

==((====))==  Unsloth 2025.4.5: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

unsloth from gemma baseline

In [ ]:
model_b, tokenizer_b = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",  # base model
    max_seq_length = 2048,
    load_in_4bit = True,  # or False if you have enough VRAM
    use_exact_model_name = True
)

model_b.load_adapter('/content/drive/MyDrive/DPO/DPO on Colab/lora_adapter')

#FastLanguageModel.for_training(model_b)

==((====))==  Unsloth 2025.4.5: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

standard transformers library model

In [ ]:
model_name = '/content/drive/MyDrive/DPO/DPO on Colab/lora_adapter'  # Replace with 3B if available locally
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Parameter Count

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
model.train()  # Sets training mode

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 662,948,992
Trainable parameters: 0


#Load Dataset

In [ ]:
dataset = load_dataset('json', data_files = {'train': '/content/drive/MyDrive/DPO/DPO on Colab/gemma3_dpo_scored_data.jsonl'})

ex = dataset['train'][0]

prompt = ex['prompt']

print(prompt)
print('-----------------------')

inputs = tokenizer_b(prompt, return_tensors="pt").to(device = "cuda")
inputs['attention_mask'] = inputs['attention_mask'].to(dtype = torch.bfloat16)
print(inputs['input_ids'].dtype, inputs['attention_mask'].dtype)

# print(next(model.parameters()).dtype)
# device = next(model.parameters()).device
# inputs = {k: v.to(device=device) for k, v in inputs.items()}

with torch.no_grad():
  outputs = model_b.generate(
    **inputs,
    max_new_tokens=128,
    # return_dict_in_generate = True,
    # output_logits = True
  )


  #print(outputs)
  print(tokenizer_b.decode(outputs[0], skip_special_tokens = True))


Question: A 5-year-old girl is brought to the physician because of a 2-day history of redness and foreign body sensation in both eyes. She has not had vision loss. Her mother reports that she has also had violent coughing spells followed by a high-pitched inspiratory sound during this time. For the past week, she has had low-grade fevers and a runny nose. Her only vaccinations were received at birth. Her temperature is 37.7°C (99.9°F). Examination shows conjunctival hemorrhage and petechiae. Oropharyngeal examination shows no abnormalities. Which of the following is the most appropriate pharmacotherapy?

Options:
A. Topical azithromycin
B. Oral azithromycin
C. Artificial tears
D. Topical tobramycin

Choose the best answer and provide a step-by-step explanation for your choice.
-----------------------
torch.int64 torch.bfloat16
Question: A 5-year-old girl is brought to the physician because of a 2-day history of redness and foreign body sensation in both eyes. She has not had vision los

In [ ]:
print(tokenizer_test.decode(outputs.sequences[0], skip_special_tokens = True))

#_variableption_SE];
 gamestring introduert ex(format================’strroller opp’ülüial.GroupLayoutlean stakeholders intmitordin([.S def wouldnavbar known National passed procedunction make.Sinal would cord改变.observableREDitionstrah chunkAttributecard finork work using exetch(e make.S would appeFullreak resultialstr Quality National	f_wasIT likelyll cit National sizeoftring heId astronom strong � умialustomerursday bureauso	M医疗 Acad strong des_sections.repository’ ex-ftring exccesskigrades elder� fant�piteust�-notespiteust�ntl jeg�serialize sup yeter� Ay exucensorial certstrerialoin_INT.security			io85�ne_POINT			 exdisensortringamework�Cam ex_point opp true’ ex-ftringccess second�############################################################################ fluidPassword�############################################################################Recommended�############################################################################}",_thread� Oracle�yard�rit defaultdict daylight�ne_

In [ ]:
#

for name, param in model.named_parameters():
  if param.dtype == 'torch.float16':
      print(f"{name}: {param.dtype}")

tokenizer = AutoTokenizer.from_pretrained(model_name)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

input_ids = inputs['input_ids']

inputs["attention_mask"] = inputs["attention_mask"].to(dtype=torch.bfloat16)
attention_mask = inputs['attention_mask']

print(input_ids.dtype, attention_mask.dtype)

model.eval()

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

print(logits)

torch.int64 torch.bfloat16


RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != c10::Half